In [74]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import sqlite3
from pprint import pprint
from typing import TypedDict, List, Annotated, Union, Dict, Any
from langchain_core.messages import BaseMessage,AIMessage,HumanMessage,SystemMessage
from pydantic import BaseModel, Field
import json
import matplotlib.pyplot as plt
from langgraph.graph import StateGraph, END, add_messages
import operator
from langchain_core.tools import tool
from datetime import datetime
import subprocess
load_dotenv()

True

In [65]:
llm_gemini = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash',temperature=0,max_retries=2)
llm_model_groq = ChatGroq(model = 'llama-3.1-8b-instant')
llm_qwen_coder = ChatOllama(model="qwen2.5-coder:1.5b")

In [66]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Mistral LLM from Ollama

llm_qwen_coder.invoke('''You are an expert Python code reviewer, you can understand the python code in a logical way
    Here is the python code:
    print(10")

    Here is the user reported issue:
    {Not able to print numeric 10}

    Base on the above issue, Think carefully: What could be causing this issue based on the logic, syntax, or control flow?
    Respond with your reasoning in 2–3 sentences.
    ''')

AIMessage(content='The error message "Not able to print numeric 10" indicates that there is an issue with the code execution. The Python interpreter attempts to convert the string "10" into a number, which is not possible because it is not a valid integer or float value. To resolve this issue, you need to ensure that the input is a valid number before attempting to print it. You can use the `isdigit()` method to check if the input contains only digits and then convert it to an integer using the `int()` function. Here\'s an example of how you can modify the code:\n```python\ndef main():\n    user_input = input("Enter a numeric value: ")\n    if user_input.isdigit():\n        print(int(user_input))\n    else:\n        print("Please enter a valid numeric value.")\n\nif __name__ == "__main__":\n    main()\n```\n\nIn this modified code, we first prompt the user to enter a numeric value using the `input()` function. We then check if the input contains only digits using the `isdigit()` method

In [67]:
class AgentState(TypedDict):

    code_history: List[BaseMessage]
    query_issue: List[BaseMessage] # General question history
    think_history: Dict[str,List[BaseMessage]]# Thoughts by agents
    act_history: Dict[str,List[BaseMessage]]# Actions taken
    observe_history: Dict[str,List[BaseMessage]] # Observations received
    reflect_history: Dict[str,List[BaseMessage]] # Reflexion checkpoints
    message_history_after_vote = Annotated[List[BaseMessage],add_messages] ##message history after voting, (will encompass user query & code gen,doc gen part)
    max_iterations: int
    code_bug_detection: Dict[str,List[str]] # e.g., {"qwen": ["off by one"], "gemini": ["loop never ends"]}
    line_localisation: Dict[str,List[int]] # e.g., {"qwen": [12, 14], "gemini": [13]}

In [70]:
class thinkNodeOut(BaseModel):
    output_response: str = Field(...,description='Paragraph with 2-3 sentences on why the code has a particular issue')
    
def thinkNode(state: AgentState):

    llm_choices = {'gemini':llm_gemini,'groq':llm_model_groq,'qwen':llm_qwen_coder}
    THINK_PROMPT = '''
    You are an expert Python code reviewer, you can understand the python code in a logical way
    Here is the python code:
    {python_code}

    Here is the user reported issue:
    {query_issue}

    Base on the above issue, Think carefully: What could be causing this issue based on the logic, syntax, or control flow?
    Respond with your reasoning in a single paragraph with 2–3 sentences.
    '''
    think_prompt_template = ChatPromptTemplate.from_template(THINK_PROMPT)
    #chain_list = {}
    for llm_model in list(llm_choices.keys()):
        print(llm_model)
        think_chain = think_prompt_template | llm_choices[llm_model].with_structured_output(thinkNodeOut)
        query_issue = state['query_issue'][-1].content
        #print(query_issue)
        code = state['code'][-1].content
        print(code)
        #print(query_issue,code)
        thought_response = think_chain.invoke({'python_code':code,'query_issue':query_issue})
        #print(thought_response)
        state['think_history'][llm_model].append(AIMessage(content=thought_response.output_response))
    
    
    return state


In [73]:
state_after_think = thinkNode({'think_history':{'gemini':[],'groq':[],'qwen':[]},'code':[HumanMessage(content='print(10")')], \
           'query_issue':[HumanMessage(content='Cannot print numeric 10')]})
state_after_think

gemini
print(10")
groq
print(10")
qwen
print(10")


{'think_history': {'gemini': [AIMessage(content='The code has a syntax error. The issue is likely due to a missing closing parenthesis which causes the interpreter to not understand the command. The correct syntax should be print(10).', additional_kwargs={}, response_metadata={})],
  'groq': [AIMessage(content='The issue is likely due to a missing closing parenthesis in the print statement, which would result in a syntax error. This would cause the interpreter to fail, preventing the numeric value 10 from being printed. The corrected code would be print(10) with an added closing parenthesis.', additional_kwargs={}, response_metadata={})],
  'qwen': [AIMessage(content='The user is trying to print the integer number 10, but instead, they receive an error message indicating that they cannot print numeric 10. This issue is likely due to incorrect syntax or missing quotes around the integer value in the `print()` function call.', additional_kwargs={}, response_metadata={})]},
 'code': [Huma

In [ ]:
##Tools###

In [ ]:
def actNode(state:AgentState):
    act_prompt = '''
    
    '''

In [86]:
import inspect
import textwrap
def buggy_code():
    print("Hello")
    print(10)

# Convert to string
code_string = textwrap.dedent(inspect.getsource(buggy_code))
code_string

'def buggy_code():\n    print("Hello")\n    print(10)\n'

In [90]:
import inspect
import textwrap
from typing import Union, List

def format_code_for_llm(
    code: Union[str, List[str], object], 
    wrap_backticks: bool = False, 
    add_line_numbers: bool = False
) -> str:
    """
    Formats Python code into a string suitable for LLM input.
    
    Args:
        code: Code as a string, list of lines, or a function/class object.
        wrap_backticks: Whether to wrap output in triple backticks.
        add_line_numbers: Whether to add line numbers for each line.
        
    Returns:
        A formatted code string.
    """
    if isinstance(code, list):
        code_str = "\n".join(code)
    elif callable(code):
        code_str = inspect.getsource(code)
    elif isinstance(code, str):
        code_str = code
    else:
        raise ValueError("Unsupported input type. Use str, list, or function/class.")

    # Dedent and clean
    code_str = textwrap.dedent(code_str).strip()
    code_lines = code_str.split("\n")

    if add_line_numbers:
        code_lines = [f"{i+1:3}: {line}" for i, line in enumerate(code_lines)]
        code_str = "\n".join(code_lines)

    if wrap_backticks:
        return f"```\n{code_str}\n```"
    return code_str
def example():
    x = 5
    print("x =", x)

format_code_for_llm(example, add_line_numbers=True)

'  1: def example():\n  2:     x = 5\n  3:     print("x =", x)'